In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data_dir = "/kaggle/input/Kannada-MNIST/"
train = pd.read_csv(data_dir + "train.csv")
train.shape

In [ ]:
test = pd.read_csv(data_dir + "test.csv")
test.shape

In [ ]:
X_train = train.iloc[:,1:]
y_train = train.iloc[:,0]
X_test = test.iloc[:,1:]

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)

In [ ]:
# Normalize pixel values to be between 0 and 1
X_train, X_test = X_train / 255.0, X_test / 255.0

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_vali, y_train, y_vali = train_test_split(X_train,
                                                   y_train,
                                                   test_size = 0.2)

print(X_train.shape, X_vali.shape, y_train.shape, y_vali.shape)

In [ ]:
def reshape(X_train): 
    X_train_reshape = []
    for i in range(len(X_train)):
        X_train_reshape.append(X_train[i].reshape(28, 28, 1))
    X_train_reshape = np.array(X_train_reshape)
    return X_train_reshape

In [ ]:
X_train_reshape = reshape(X_train)
X_vali_reshape = reshape(X_vali)
X_test_reshape = reshape(X_test)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_vali = to_categorical(y_vali)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

from tensorflow.keras import datasets, layers, models

from tensorflow.keras.optimizers import Adam

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range = 0.1,
    width_shift_range=0.1,
    height_shift_range=0.1)

In [ ]:
datagen.fit(X_train_reshape)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience = 3,
                              verbose = 1, min_lr=0.00001)

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(filters = 32, kernel_size = (5, 5), 
                        activation='relu', 
                        padding = 'same', 
                        input_shape=(28, 28, 1)))
model.add(layers.Conv2D(filters = 32, kernel_size = (5, 5), 
                        activation='relu', 
                        padding = 'same'))
model.add(layers.BatchNormalization(momentum = 0.1))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(filters = 64, kernel_size = (3, 3), 
                        activation='relu', 
                        padding = 'same'))
model.add(layers.Conv2D(filters = 64, kernel_size = (3, 3), 
                        activation='relu', 
                        padding = 'same'))
model.add(layers.BatchNormalization(momentum = 0.1))
model.add(layers.MaxPooling2D((2,2), strides=(2,2)))
model.add(layers.Dropout(0.2))

model.add(layers.Conv2D(filters = 32, kernel_size = (5, 5), 
                        activation='relu', 
                        padding = 'same'))
model.add(layers.Conv2D(filters = 32, kernel_size = (5, 5), 
                        activation='relu', 
                        padding = 'same'))
model.add(layers.BatchNormalization(momentum = 0.1))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Dropout(0.2))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

In [ ]:
optimizer=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

model.compile(optimizer=optimizer,
          loss=['categorical_crossentropy'],
          metrics=['accuracy'])

In [ ]:
batch_size = 64
epochs = 30

history = model.fit(datagen.flow(X_train_reshape, y_train), epochs=epochs, batch_size = batch_size,
                    validation_data=(X_vali_reshape, y_vali), verbose = 2, 
                    steps_per_epoch = X_vali.shape[0]//batch_size,
                    callbacks=[reduce_lr])

In [ ]:
def plot_accuracy(history):
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    # plt.ylim([0.5, 1])
    plt.legend(loc='lower right')

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
# plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
predicted = model.predict(X_test_reshape)

In [ ]:
predicted = np.argmax(predicted, axis = 1)

In [ ]:
index = np.array(test.index)
res = pd.DataFrame(np.concatenate([index[:,np.newaxis],predicted[:,np.newaxis]], axis = 1), columns = ['id', 'label'])


In [ ]:
res

In [ ]:
res.to_csv('submission.csv', index = False)